<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

### Do this with the "premade" code in lecture, then try to use Keras I think

In [15]:
import os
import numpy as np

Get the text from file

In [17]:
readin_work = []
data_path = './fancyboi/work.txt'
with open(data_path, 'r') as f:
    content = f.read()
    readin_work.append(content)

Do some misc things to take a looky-loo at the data

In [20]:
writtens = readin_work
text = " ".join(writtens)
chars = list(set(text))
chars_amt = len(chars)
text_size = len(text)

In [24]:
print(f'''
Unique characters: {chars_amt}
Size of the text: {text_size} (idk if in bytes or whatever)
''')


Unique characters: 107
Size of the text: 5584464 (idk if in bytes or whatever)



So, supposed to one hot encode all the characters in question...oh boi

In [31]:
parasite_c2i = dict((c, i) for i, c in enumerate(chars))  # Allegedly enumerate returns index & value. Converts to dict(?)
parasite_i2c = dict((i, c) for i, c in enumerate(chars))
# Integer encode inputs?
int_in_rock = [parasite_c2i[i] for i in text]  # List which has sequence converted from OG data to integers

In [33]:
print(f'''
Uno: {parasite_c2i}\n {'-'*30}
Dos: {parasite_i2c}\n {'-'*30}
Data length: {len(int_in_rock)}...again idk if this is bytes or whatever
''')


Uno: {'.': 0, '3': 1, 'œ': 2, 'Q': 3, '\ufeff': 4, '[': 5, 'A': 6, "'": 7, '%': 8, 'x': 9, 'N': 10, 'Æ': 11, 'O': 12, 'v': 13, 'î': 14, 'r': 15, 'g': 16, '5': 17, '|': 18, '-': 19, 'é': 20, '2': 21, '#': 22, 'V': 23, '\t': 24, '\\': 25, ':': 26, '$': 27, 'k': 28, '&': 29, 'd': 30, 'q': 31, '8': 32, '4': 33, ']': 34, 'F': 35, 'X': 36, '\n': 37, 'è': 38, '—': 39, 'D': 40, 'U': 41, 'W': 42, 'R': 43, '*': 44, 's': 45, 'n': 46, '/': 47, '?': 48, 'K': 49, 't': 50, ';': 51, 'z': 52, '’': 53, 'u': 54, 'E': 55, 'o': 56, 'a': 57, '}': 58, 'f': 59, 'â': 60, 'É': 61, '_': 62, 'M': 63, 'P': 64, 'j': 65, 'b': 66, '1': 67, '”': 68, 'L': 69, '6': 70, 'G': 71, 'y': 72, ',': 73, 'e': 74, 'Y': 75, ')': 76, 'H': 77, 'I': 78, 'S': 79, 'w': 80, 'ç': 81, '0': 82, 'T': 83, '9': 84, '!': 85, '@': 86, 'l': 87, 'c': 88, ' ': 89, 'B': 90, 'J': 91, 'æ': 92, 'Z': 93, 'à': 94, 'p': 95, '(': 96, '“': 97, 'm': 98, 'ê': 99, 'i': 100, '`': 101, 'h': 102, '7': 103, '"': 104, '‘': 105, 'C': 106}
 ------------------------

#### Now onto MANUALLY doing this crazy code...

In [80]:
# Hyperparam pen
iterations = 1000
seq_length = 40
batch_size = round((text_size / seq_length) + 0.5)  # this is math.ceil basically
hidden_sze = 500  # Size of hidden layer of neurons
learning_rate = 1e-1

In [81]:
# Model parameters
weight_in_hid = np.random.randn(hidden_sze, chars_amt) * 0.01  # Weight INPUT -> hidden
weight_hid_hid = np.random.randn(hidden_sze, hidden_sze) * 0.01  # Weight hidden -> hidden
weight_hid_out = np.random.randn(chars_amt, hidden_sze) * 0.01  # Weight hidden -> OUTPUT
# Bias stuff
bias_hid = np.zeros((hidden_sze, 1))  # Hidden bias
bias_out = np.zeros((chars_amt, 1))  # Output bias
# Think this is hidden t-1, like obvi the previous one but yeah (yeah)
hid_prev = np.zeros((hidden_sze, 1))  # h_(t-1) idk what this is but the teacher wrote it...

#### Forward Prop!

In [82]:
def forwardProp(inputs, targets, hid_prev):
    
    # Since RNN receives sequence, weights are not updated during 1 sequence (idk what this means)
    in_s, hid_s, out_s, prob_s = {}, {}, {}, {}
    hid_s[-1] = np.copy(hid_prev)  # Copy prev hidden state vekkie to -1 key value
    # Loss init
    loss = 0
    
    # For loop to go through time, woooo
    for t in range(len(inputs)):
        # t is lookup value (key)
        # Sets INPUT state into something?
        in_s[t] = np.zeros((chars_amt,1))
        # Make INPUT sequence's input keyed values into 1 (?)
        in_s[t][inputs[t]] = 1
        # This sets the HIDDEN state
        hid_s[t] = np.tanh(np.dot(weight_in_hid, in_s[t]) +
                           np.dot(weight_hid_hid, hid_s[t-1]) +
                           bias_hid)
        # Sets OUT state; unnormalized log probs for next chars (?)
        out_s[t] = (np.dot(weight_hid_out, hid_s[t]) +
                    bias_out)
        # PROB for next chars
        prob_s = (np.exp(out_s[t]) /
                  np.sum(np.exp(out_s[t])))
        
        # Softmax time! AKA: normalizing a vector of real numbers proportional to the exponentials of the input nums
        loss += -np.log(prob_s[t][targets[t], 0])  # (cross-entropy loss)
        
    return loss, prob_s, hid_s, in_s

Wow! That wasn't fun, let's do something harder!

#### Backward Prop!

In [83]:
def backProp(prob_s, inputs, hid_s, in_s, targets):
    
    # Oh boy here comes these absolutely inhuman variable names.
    # So ugly in fact, that I can't even rename them cause they're hieroglyphs and I have a life.
    ### stinkzone
    # Make all zero matrices
    dWxh = np.zeros_like(weight_in_hid)
    dWhh = np.zeros_like(weight_hid_hid)
    dWhy = np.zeros_like(weight_hid_out)  # Yea dWhy is right
    dbh = np.zeros_like(bias_hid)
    dby = np.zeros_like(bias_out)
    dhnext = np.zeros_like(hid_s[0]) # (hidden_size,1) 
    ### Look at this stuff and tell me these variable names ain't crazy...cmonnnn
    
    # Reversed
    for t in reversed(range(len(inputs))):
        dy = np.copy(prob_s[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
        dy[targets[t]] -= 1 # backprop into y. After taking the soft max in the input vector, subtract 1 from the value of the element corresponding to the correct label.
        dWhy += np.dot(dy, hid_s[t].T)
        dby += dy 
        dh = np.dot(weight_hid_out.T, dy) + dhnext # backprop into h. 
        dhraw = (1 - hid_s[t] * hid_s[t]) * dh # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
        dbh += dhraw
        dWxh += np.dot(dhraw, in_s[t].T)
        dWhh += np.dot(dhraw, hid_s[t-1].T)
        dhnext = np.dot(weight_hid_hid.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.  
    
    return dWxh, dWhh, dWhy, dbh, dby

Totally not monstrous.

#### Training!

In [84]:
%%time

data_pointer = 0

# Mem variables for Adagrad (?) LOOK AT THESE NAMES CMON
mWxh = np.zeros_like(weight_in_hid)
mWhh = np.zeros_like(weight_hid_hid)
mWhy = np.zeros_like(weight_hid_out)  # Yeah mWhy..
mbh, mby = np.zeros_like(bias_hid), np.zeros_like(bias_out)

# A giant for loop!
for i in range(iterations):
    hid_prev = np.zeros((hidden_sze, 1))  # Reset RNN memory
    data_pointer = 0  # Zero so that you can go from start of data
    
    # ---------------------------------------------------------------------------------
    # Somewhere here we need to change something to get this working with good ole' Shakespeare
    for b in range(batch_size):
        inputs = [parasite_c2i[ch] 
                  for ch in text[
                      data_pointer:data_pointer+seq_length
                  ]
                 ]
        targets = [parasite_c2i[ch] 
                  for ch in text[
                      data_pointer+1:data_pointer+seq_length+1
                  ]
                 ]
        # Now we apparently process last part of input data.
        if (data_pointer+seq_length+1 >= len(text) and
            b == batch_size-1):
            targets.append(parasite_c2i[" "])  # When data doesn't fit, add space (" ") to the back (?)
        
        # Forward prop!
        loss, prob_s, hid_s, in_s = forwardProp(inputs, targets, hid_prev)
        # Backward prop!
        dWxh, dWhh, dWhy, dbh, dby = backProp(prob_s, inputs, hid_s, in_s, targets)
        
        # Parameter update with Adagrad (again with this word)
        # What. In. God's. Green. Earth. Is. This.
        for param, dparam, mem in zip([weight_in_hid, weight_hid_hid,
                                       weight_hid_out, bias_hid, bias_out],
                                      [dWxh, dWhh, dWhy, dbh, dby],
                                      [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam  # Elementwise multiplication I guess
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # Adagrad update
        
        data_pointer += seq_length  # Move the data pointer again...
    # ---------------------------------------------------------------------------------
    # For every 100th iteration, print some progress
    if i % 100 == 0:
        print(f'Iteration: {i}\nLoss: {loss}')

IndexError: too many indices for array

There is a bug somewhere but there's no way I'm debugging this anymore. I managed to make it work on the OG lecture notebook and it's...slow, so I'm going to move onto Keras to do non-manual stuff
![wah](a.png)

In [85]:
def predict(test_char, length):
    x = np.zeroes((chars_amt, 1))
    x[parasite_c2i[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_sze, 1))
    
    for t in range(length):
        h = np.tanh(np.dot(weight_in_hid, x) + np.dot(weight_hid_hid, h) + bias_hid) 
        y = np.dot(weight_hid_out, h) + bias_out
        p = np.exp(y) / np.sum(np.exp(y)) 
        ix = np.random.choice(range(chars_amt), p=p.ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        x = np.zeros((chars_amt, 1)) # init
        x[ix] = 1 
        ixes.append(ix) # list
    txt = test_char + ''.join(parasite_i2c[i] for i in ixes)
    print(f'\b~~~{txt}')

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN

## Keras!

All cells below ripped from [mlm](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/)

In [100]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "./fancyboi/work.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 50
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=2, batch_size=48, callbacks=callbacks_list)

Epoch 1/2
 340224/5584414 [>.............................] - ETA: 5:59:52 - loss: 2.8137

KeyboardInterrupt: 

In [95]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "./fancyboi/work.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
filename = "weights-improvement-.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Total Characters:  5584464
Total Vocab:  79
Total Patterns:  5584364


OSError: Unable to open file (unable to open file: name = 'weights-improvement-47-1.2219-bigger.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)